In [19]:
import pandas as pd
import urllib.parse as urlparse
import json
import re
import requests
import math
import time
from datetime import datetime, date
from os.path import exists
bridge_file = open('./Bridge.txt','r')
key_list = bridge_file.readlines()
YT_API_KEY = key_list[0]
PERS_API_KEY = key_list[1]
bridge_file.close()

In [2]:
def dropRowFrom(input_dataset,row_number):
    input_dataset = time_df.drop(row_number)
    input_dataset.reset_index(drop=True, inplace=True)
    return input_dataset

In [3]:
def startTimeMe(print_flag):
    now = datetime.now()
    dt_string = now.strftime("%H:%M:%S")
    current_time = time.time()
    if print_flag:
        print(f'Start time: {dt_string}')
    #time_list = dt_string.split(':')
    #time_hr,time_min, time_sec = time_list[0],time_list[1],time_list[2]
    return dt_string,current_time

In [4]:
def endTimeMe(input_dataset, label, label_for, nums, dt_string_s, start_time):
    dt_string_e,current_time = startTimeMe(False)
    print(f'End time: {dt_string_e}')
    #print(time_hr,time_min, time_sec)
    input_dataset.loc[len(input_dataset.index)] = [label,label_for,nums,int(current_time)-int(start_time),
                                                   dt_string_s,dt_string_e,start_time,current_time
                                                   ]
    return dt_string_e,current_time,input_dataset

In [5]:
def getCommentsForVideo(video_id):
    comment_list = []
    count = 0
    comments_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId={video_id}&key={YT_API_KEY}')
    comment_json_data = json.loads(comments_from.text)
    if 'items' in comment_json_data:
        for x in range(0,len(comment_json_data['items'])):
            comment_element = {
                'id': comment_json_data['items'][x]['id'],
                'textOriginal': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['textOriginal'],
                'authorDisplayName': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'likeCount': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['updatedAt']
            }
            comment_list.append(comment_element)
        count = count+len(comment_json_data['items'])
        print(f'{count} processed',end="\r")
        while('nextPageToken' in comment_json_data):
            next_page_token = comment_json_data['nextPageToken']
            comments_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&pageToken={next_page_token}&videoId={video_id}&key={YT_API_KEY}')
            comment_json_data = json.loads(comments_from.text)
            if 'items' in comment_json_data:
                for x in range(0,len(comment_json_data['items'])):
                    comment_element = {
                        'id': comment_json_data['items'][x]['id'],
                        'textOriginal': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['textOriginal'],
                        'authorDisplayName': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                        'likeCount': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['likeCount'],
                        'publishedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['publishedAt'],
                        'updatedAt': comment_json_data['items'][x]['snippet']['topLevelComment']['snippet']['updatedAt']
                    }
                    comment_list.append(comment_element)
                count = count+len(comment_json_data['items'])
                print(f'{count} processed',end="\r")    
    return count,len(comment_list), comment_list
#getCommentsForVideo('quOHHVVYWlo')

In [6]:
def get_topic_category_from_wiki(wiki_link):
    if 'en.wikipedia.org/wiki/' in wiki_link:
        topic_index = wiki_link.index('wiki/')
        return wiki_link[topic_index+5:]
    else:
        return 'NA'

In [7]:
def convert_time_to_sec(time_str):
    #PT1H15M44S
    total_seconds = 0
    
    if 'DT' in time_str:
        time_str = time_str[1:]
        index = time_str.index('DT')
        day_time = int(time_str[0:index])
        total_seconds = day_time*86400
        time_str = time_str[index+2:]
    else:
        time_str = time_str[2:]
    if 'H' in time_str:
        index = time_str.index('H')
        hour_time = int(time_str[0:index])
        total_seconds = total_seconds+(hour_time*3600)
        time_str = time_str[index+1:]
        #print(hour_time)
        #print(time_str)
    if 'M' in time_str:
        index = time_str.index('M')
        min_time = int(time_str[0:index])
        total_seconds = total_seconds+(min_time*60)
        time_str = time_str[index+1:]
        #print(time_str)
        #print(min_time)
    if 'S' in time_str:
        index = time_str.index('S')
        sec_time = int(time_str[0:index])
        total_seconds = total_seconds+sec_time
        #print(sec_time)
    return total_seconds

In [8]:
def get_video_category_from_id(categoryId):
    category_dict = {'1':'Film & Animation','2':'Autos & Vehicles','10':'Music','15':'Pets & Animals','17':'Sports','19':'Travel & Events',
'20':'Gaming','22':'People & Blogs','23':'Comedy','24':'Entertainment','25':'News & Politics','26':'Howto & Style','27':'Education',
'28':'Science & Technology','29':'Nonprofits & Activism'}
    if categoryId in category_dict:
        return category_dict[categoryId]
    else:
        print(f'{categoryId}: NA')
        return 'NA'

In [9]:
def countUrlType(input_dataset):
    channel_count = 0
    video_count = 0
    other_count = 0
    for idx in input_dataset.index:
        if input_dataset['url_type'][idx] == 'channel':
            channel_count=channel_count+1
        elif input_dataset['url_type'][idx] == 'video':
            video_count=video_count+1
        else:
            other_count=other_count+1
    return channel_count,video_count,other_count

In [10]:
def expanded_url_from(urllist_item):
    urllist_item = str(urllist_item)
    if 'nan' in urllist_item:
        return "NA"
    if 'expanded_url' in urllist_item:
        if 'youtube.com' not in urllist_item and 'youtu.be' not in urllist_item:
            print("Check this link: "+urllist_item)
            return "NA"
        index_one = urllist_item.index('expanded_url')+16
        ext_url = ''
        for x in range(index_one,len(urllist_item)):
            if urllist_item[x] == '\'':
                return ext_url
            else:
                ext_url = ext_url+urllist_item[x]
        return urllist_item[index_one:index_two]
    else:
        return "NA"
#expanded_url_from("[{'url': 'https://t.co/cZdjZpdrGh', 'expanded_url': 'https://youtu.be/ymdrOPAzLqc', 'display_url': 'youtu.be/ymdrOPAzLqc', 'indices': [169, 192]}]")

In [11]:
def checkSuccess(ch_count, vi_count, ot_count, sup_total):
    if (ch_count+vi_count+ot_count) == sup_total:
        return True
    else:
        return False

In [12]:
def getIDfromURL(yt_url):
    link_type = ''
    link_id = ''
    if '/c/' in yt_url or '/channel/' in yt_url:
        if '/c/' in yt_url:
            c_index = yt_url.index('/c/')
            sub_yt_url = yt_url[c_index+3:]
            if '/' in sub_yt_url:
                slash_index = sub_yt_url.index('/')+c_index+3
                link_type = 'channel'
                link_id = yt_url[c_index+3:slash_index]
            elif '?' in sub_yt_url:
                quest_index = sub_yt_url.index('?')+c_index+3
                link_type = 'channel'
                link_id = yt_url[c_index+3:quest_index]                            
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+3:]
            if len(link_id) != 24:
                #make a channel id request
                channelid_response = requests.get(f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=channel&fields=items%2Fsnippet%2FchannelId&q={link_id}&key={YT_API_KEY}')
                channelid_json = json.loads(channelid_response.text)
                if 'items' in channelid_json and len(channelid_json['items']) > 0:
                    if pd.isna(channelid_json['items'][0]['snippet']['channelId']):
                        print(f'true NA {link_id}')
                        link_id='NA'
                    else:
                        link_id = channelid_json['items'][0]['snippet']['channelId']
                else:
                    print(f'{channelid_json}')
                    link_id='NA'
        elif '/channel/' in yt_url:
            c_index = yt_url.index('/channel/')
            sub_yt_url = yt_url[c_index+9:]
            if '/' in sub_yt_url:
                slash_index = sub_yt_url.index('/')+c_index+9
                link_type = 'channel'
                link_id = yt_url[c_index+9:slash_index]
            elif '?' in sub_yt_url:
                quest_index = sub_yt_url.index('?')+c_index+9
                link_type = 'channel'
                link_id = yt_url[c_index+9:quest_index]  
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+9:]
    elif '/watch' in yt_url or '/v=' in yt_url or '/shorts/' in yt_url or '.be/' in yt_url:
        if '/watch' in yt_url:
            w_index = yt_url.index('v=')
            link_type = 'video'
            link_id = yt_url[w_index+2:w_index+13]
        elif '/v=' in yt_url:
            w_index = yt_url.index('/v=')
            link_type = 'video'
            link_id = yt_url[w_index+3:w_index+14]
        elif '/shorts/' in yt_url:
            w_index = yt_url.index('/shorts/')
            link_type = 'video'
            link_id = yt_url[w_index+8:w_index+19]  
        elif '.be/' in yt_url:
            w_index = yt_url.index('.be/')
            link_type = 'video'
            link_id = yt_url[w_index+4:w_index+15]
    else:
        print(yt_url)
        com_index = yt_url.index('com/')
        if (len(yt_url)-(com_index+4)) == 11:
            link_type = 'video'
            link_id = yt_url[com_index+4:com_index+15]
        else:
            link_type = 'other'
            link_id = 'NA'
    return link_type,link_id

In [13]:
def takeDateQuery():
    month_dict = {'01': 'Jan','02': 'Feb','03': 'Mar','04': 'Apr',
             '05': 'May','06': 'Jun','07': 'Jul','08': 'Aug',
             '09': 'Sep','10': 'Oct','11': 'Nov','12': 'Dec'}
    date_query = input('Input date in MM/DD/YYYY format').split('/')
    month_query = month_dict[date_query[0]]
    day_query = date_query[1]
    year_query = date_query[2]
    print(month_query,day_query,year_query)
    return month_query,day_query,year_query

In [23]:
#Initializing time statistics dataframe

In [24]:
time_df = pd.DataFrame(columns=['Activity','Type','Quantity','Time Taken (sec)','Start Time','End Time','Start (sec)','End (sec)'])

In [16]:
dt_string_s,start_time = startTimeMe(True)
twitter_dataset_onlyurl = pd.read_csv('./datasets/yt_onlyurl.csv')
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Reading \'yt_onlyurl\' dataset','Rows', len(twitter_dataset_onlyurl),
                                             dt_string_s,start_time)
time_df

Start time: 16:16:36
End time: 16:16:39


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09


In [49]:
twitter_dataset_onlyurl.columns

Index(['tweetid', 'urls_list', 'url'], dtype='object')

In [47]:
twitter_dataset_onlyurl.size

3908106

In [15]:
dt_string_s,start_time = startTimeMe(True)
twitter_dataset = pd.read_csv('./datasets/yturl_twitter.csv',lineterminator="\n",low_memory=False)
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Reading \'yturl_twitter\' dataset','Rows',len(twitter_dataset.index),
                                             dt_string_s,start_time)
time_df

Start time: 16:15:36
End time: 16:16:19


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09


In [17]:
twitter_dataset

,tweetid,userid,screen_name,date,lang,location,description,place_id,place_url,place_type,...,state,country,rt_state,rt_country,qtd_state,qtd_country,norm_country,norm_rt_country,norm_qtd_country,acc_age
0,1495977545967317000,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:35 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,1495977557505753089,2821163219,Billington_Book,Tue Feb 22 04:23:38 +0000 2022,en,"Reus, Tarragonna, Spain","Disabled Vietnam vet, dad, granddad, cancer su...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2691
2,1495977563197476865,294162188,ArsenicMarko,Tue Feb 22 04:23:39 +0000 2022,en,"Aurora, Ontario",NaN,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,3944
3,1495977582155685895,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:44 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
4,1495977589151830017,1378410853314203649,Denise07190935,Tue Feb 22 04:23:45 +0000 2022,nl,NaN,🧠zelfnadenken \n🕊free Humanity \n📚boeken \nexi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302697,1509017366180634626,1331144263657287680,Marek82085289,Wed Mar 30 03:59:10 +0000 2022,en,NaN,PO = KORUPCJA i ZDRAJCY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490
1302698,1509017394299260931,1045050364158853120,life_canbeshort,Wed Mar 30 03:59:17 +0000 2022,en,Canada,The Universe Corrects All Wrongs✌️ pls dont sp...,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,1280
1302699,1509017428831145984,53870131,gusbayu,Wed Mar 30 03:59:25 +0000 2022,in,dipermukaan bumi,Ragadi musuh mepareng \nRihati ya tongwanya ta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4650
1302700,1509017469758832640,777674879701254144,Online_SergioJu,Wed Mar 30 03:59:35 +0000 2022,es,"Santiago, Chile",Saludos ! aquí verás él Mundo a través de mis...,NaN,NaN,NaN,...,NaN,Chile,NaN,NaN,NaN,NaN,Chile,NaN,NaN,2018


In [43]:
#Testing

In [ ]:
list(twitter_dataset['tweet_type'])

In [ ]:
for idx in twitter_dataset.index:
    print(twitter_dataset['tweet_type'][idx],twitter_dataset['rt_urls_list'][idx])

In [50]:
#Checking date format given in dataset

In [8]:
print('Date format:',twitter_dataset['date'][0].strip())
month_dict = {}
for idx in twitter_dataset.index:
    temp = twitter_dataset['date'][idx].split(' ')[1]
    if temp not in month_dict:
        month_dict[temp] = 1
print(month_dict)

Date format: Tue Feb 22 04:23:35 +0000 2022
{'Feb': 1, 'Mar': 1, 'Apr': 1}


In [53]:
#Input date from user

In [19]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [54]:
#Compiling urls tweet/retweet from dataset

In [17]:
dt_string_s,start_time = startTimeMe(True)
tweetid_urldata = []
urllist_urldata = []
for idx in twitter_dataset.index:
    tweetid_urldata.append(twitter_dataset['tweetid'][idx])
    if twitter_dataset['tweet_type'][idx] == 'retweeted_tweet_without_comment':
        urllist_urldata.append(expanded_url_from(twitter_dataset['rt_urls_list'][idx]))
    else:
        urllist_urldata.append(expanded_url_from(twitter_dataset['urls_list'][idx]))
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Compiling urls tweet/retweet from twitter dataset','URLs', len(urllist_urldata),
                                             dt_string_s,start_time)
time_df

Start time: 16:16:50
End time: 16:17:17


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09


In [55]:
#Confirming length of url list and dataset size

In [31]:
len(urllist_urldata)

1302702

In [34]:
#Extracting attributes, url ids and classifying url types from twitter dataset on the requested date

In [22]:
tweetid_list = []
userid_list = []
screenname_list = []
date_list = []
url_list = []
url_id_list = []
url_type_list = []
count_ids = 0
count_cids = 0
count_nonids = 0
count = 0
dt_string_s,start_time = startTimeMe(True)
#tweetid, userid, screen_name, date, url, url_id, url_type
for idx in twitter_dataset.index:
    date_from_dataset = twitter_dataset['date'][idx].split(' ')
    month_from_dataset = date_from_dataset[1]
    day_from_dataset = date_from_dataset[2]
    year_from_dataset = date_from_dataset[5]
    if month_from_dataset == month_query and day_from_dataset == day_query and year_from_dataset == year_query:
        tweet_id = twitter_dataset['tweetid'][idx]
        if tweet_id in tweetid_urldata:
            tweetid_index = tweetid_urldata.index(tweet_id)
            yt_url = urllist_urldata[tweetid_index]
            tweetid_list.append(tweet_id)
            userid_list.append(twitter_dataset['userid'][idx])
            screenname_list.append(twitter_dataset['screen_name'][idx])
            date_list.append(twitter_dataset['date'][idx])
            url_list.append(yt_url)
            link_type, link_id = getIDfromURL(yt_url)
            url_type_list.append(link_type)
            url_id_list.append(link_id)
            if link_type == 'other':
                count_nonids = count_nonids+1
            elif link_type == 'channel':
                count_cids = count_cids+1
            else:
                count_ids = count_ids+1
        count=count+1
        print(f'{count}/{len(twitter_dataset)} processed',end="\r")
dated_twitterset = pd.DataFrame({'tweetid': tweetid_list, 
                                 'userid': userid_list, 
                                 'screen_name': screenname_list, 
                                 'date': date_list,
                                 'url_id': url_id_list,
                                 'url':url_list, 
                                 'url_type':url_type_list })
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Extracting attributes for a date from twitter dataset','Rows', len(dated_twitterset.index),
                                             dt_string_s,start_time)

Start time: 16:18:58
{'items': []}processed
{'items': []} processed
End time: 16:19:53essed


In [23]:
time_df

,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09


In [29]:
dated_twitterset

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [56]:
#Verifying if all possible ids are extracted from the date queried dataset

In [125]:
count_cids,count_ids,count_nonids = countUrlType(dated_twitterset)
print(count_cids,count_ids,count_nonids)
checkSuccess(count_cids,count_ids,count_nonids, len(dated_twitterset))

118 17828 69


True

In [24]:
dt_string_s,start_time = startTimeMe(True)
dated_twitterset.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv',index=False)
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Saving dated twitter data as CSV','URLs', len(dated_twitterset.index),
                                             dt_string_s,start_time)
time_df

Start time: 16:20:30
End time: 16:20:30


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09


In [70]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [25]:
dt_string_s,start_time = startTimeMe(True)
dated_twitterset = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv')
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Reading dated twitter data from CSV','URLs', len(dated_twitterset.index),
                                             dt_string_s,start_time)
time_df

Start time: 16:20:35
End time: 16:20:35


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09


In [148]:
dated_twitterset

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [7]:
#Segregating different types of IDs for channel, videos and other

In [26]:
video_id_list = []
channel_id_list = []
other_id_list = []
dt_string_s,start_time = startTimeMe(True)
for idx in dated_twitterset.index:
    if dated_twitterset['url_type'][idx] == 'channel':
        channel_id_list.append(dated_twitterset['url_id'][idx])
    elif dated_twitterset['url_type'][idx] == 'video':
        video_id_list.append(dated_twitterset['url_id'][idx])
    else:
        other_id_list.append(dated_twitterset['url_id'][idx]) 
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Segregating ID types from dated twitter data','ID types', len(dated_twitterset.index),
                                             dt_string_s,start_time)
time_df

Start time: 16:20:46
End time: 16:20:46


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09


In [153]:
checkSuccess(len(channel_id_list),len(video_id_list),len(other_id_list), len(dated_twitterset))

True

In [108]:
#Gathering unique video ids from the list

In [27]:
dt_string_s,start_time = startTimeMe(True)
videoid_set = set(video_id_list)
video_id_list = list(videoid_set)
channelid_set = set(channel_id_list)
channel_id_list = list(channelid_set)
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Gathering unique video ids from the list','Unique IDs', (len(channel_id_list)+len(video_id_list)),
                                             dt_string_s,start_time)
time_df

Start time: 16:20:53
End time: 16:20:53


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09


In [110]:
#Generating comma separated list of video ids to curb YouTube Data API calls

In [28]:
dt_string_s,start_time = startTimeMe(True)
nums_video = len(video_id_list)
vid_req_num = int(nums_video/50)
videoid_index = 0
cs_videoids = ''
vid_req_list = []
for ind in range(0,vid_req_num):
    for s_ind in range(videoid_index,videoid_index+50):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+50
if nums_video % 50 != 0:
    diff = nums_video - (50*vid_req_num)
    for s_ind in range(videoid_index,videoid_index+diff):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+diff
print(f'No. of unique videos {videoid_index}, no of API calls needed: {len(vid_req_list)}') 
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Generating comma separated list of video ids','Unique IDs', videoid_index,
                                             dt_string_s,start_time)
time_df   

Start time: 16:20:59
No. of unique videos 6421, no of API calls needed: 129
End time: 16:20:59


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
9,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09


In [112]:
#Generating comma separated list of channel ids to curb YouTube Data API calls

In [29]:
dt_string_s,start_time = startTimeMe(True)
nums_channel = len(channel_id_list)
ch_req_num = int(nums_channel/50)
channelid_index = 0
cs_channelids = ''
cid_req_list = []
for ind in range(0,ch_req_num):
    for s_ind in range(channelid_index,channelid_index+50):
        if channel_id_list[s_ind] != 'NA' and str(channel_id_list[s_ind]) != 'nan':
            print(str(channel_id_list[s_ind]),end="\r")
            cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+50
if nums_channel % 50 != 0:
    diff = nums_channel - (50*ch_req_num)
    for s_ind in range(channelid_index,channelid_index+diff):
        if channel_id_list[s_ind] != 'NA' and str(channel_id_list[s_ind]) != 'nan':
            print(str(channel_id_list[s_ind]),end="\r")
            cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+diff
print(f'No of channels {channelid_index}, no of API calls needed: {len(cid_req_list)}')
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Generating comma separated list of channel ids','Unique IDs', channelid_index,
                                             dt_string_s,start_time)
time_df

Start time: 16:21:06
No of channels 29, no of API calls needed: 1
End time: 16:21:06


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
9,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09


In [81]:
#Making API calls for fetching video metadata

In [30]:
dt_string_s,start_time = startTimeMe(True)
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[0]}&key={YT_API_KEY}')
json_data = json.loads(response_from.text)
if 'items' in json_data:
    for x in range(1,len(vid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[x]}&key={YT_API_KEY}')
        json_data_2 = json.loads(response2_from.text)
        if 'items' in json_data_2:
            json_data['items'] = [*json_data['items'],*json_data_2['items']]
        print(f'{x}/{len(vid_req_list)} processed',end="\r")
    print(len(json_data['items']))
else:
    print('some error occured fetching data from API')
    print(json_data)
dt_string_e,current_time,time_df = endTimeMe(time_df, 'API calls for fetching video metadata','Videos', videoid_index,
                                             dt_string_s,start_time)
time_df

Start time: 16:21:26
5827129 processed
End time: 16:22:26


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
9,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09


In [31]:
dt_string_s,start_time = startTimeMe(True)
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics&id={cid_req_list[0]}&key={YT_API_KEY}')
channel_json_data = json.loads(response_from.text)
if 'items' in channel_json_data:
    for x in range(1,len(cid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={cid_req_list[x]}&key={YT_API_KEY}')
        channel_json_data_2 = json.loads(response2_from.text)
        if 'items' in channel_json_data_2:
            channel_json_data['items'] = [*channel_json_data['items'],*channel_json_data_2['items']]
        print(f'{x}/{len(cid_req_list)} processed',end="\r")
    print(len(channel_json_data['items']))
else:
    print('some error occured fetching data from API')
    print(channel_json_data)
dt_string_e,current_time,time_df = endTimeMe(time_df, 'API calls for fetching channel metadata','Channels', channelid_index,
                                             dt_string_s,start_time)
time_df

Start time: 16:22:47
28
End time: 16:22:47


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
9,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09


In [84]:
#Extracting metadata from YouTube video JSON

In [32]:
dt_string_s,start_time = startTimeMe(True)
v_url = []
v_channelId = []
v_channelTitle = []
v_tags=[]
v_views=[]
v_likes=[]
v_title=[]
v_lang = []
v_category = []
v_topic_categories = []
v_description=[]
v_publish_date=[]
v_comments=[]
v_videoTime = []
v_visited = []
v_contentRating = []
v_comments = []
unavailable_videoIds = []
response_id_list = []
data_list = json_data['items']
count = 0
for item in data_list:
    response_id_list.append(item['id'])
for id in video_id_list:
    v_url.append('https://www.youtube.com/watch?v='+id)
    if id not in response_id_list:
        v_channelTitle.append('NA')
        v_channelId.append('NA')
        v_tags.append(['NA'])
        v_lang.append('NA')
        v_category.append('NA')
        v_videoTime.append(0)
        v_views.append(0)
        v_likes.append(0)
        v_topic_categories.append(['NA'])
        v_comments.append(0)
        v_title.append('NA')
        v_description.append('NA')
        v_publish_date.append('NA')
        v_contentRating.append('NA')
    else:
        v_index = response_id_list.index(id)
        if 'channelTitle' in json_data['items'][v_index]['snippet']:
            v_channelTitle.append(json_data['items'][v_index]['snippet']['channelTitle'])
        else:
            v_channelTitle.append('NA')
        if 'channelId' in json_data['items'][v_index]['snippet']:
            v_channelId.append(json_data['items'][v_index]['snippet']['channelId'])
        else:
            v_channelId.append('NA')
        if 'tags' in json_data['items'][v_index]['snippet']:
            v_tags.append(json_data['items'][v_index]['snippet']['tags'])
        else:
            v_tags.append(['NA'])
        if 'defaultLanguage' in json_data['items'][v_index]['snippet']:
            v_lang.append(json_data['items'][v_index]['snippet']['defaultLanguage'][0:2])
        else:
            v_lang.append('NA')
        if 'categoryId' in json_data['items'][v_index]['snippet']:
            v_category.append(get_video_category_from_id(json_data['items'][v_index]['snippet']['categoryId']))
        else:
            v_category.append('NA')
        if 'duration' in json_data['items'][v_index]['contentDetails']:
                #print(item['id'])
            v_videoTime.append(convert_time_to_sec(json_data['items'][v_index]['contentDetails']['duration']))
        else:
            v_videoTime.append(0)
        if 'viewCount' in json_data['items'][v_index]['statistics']:
            v_views.append(int(json_data['items'][v_index]['statistics']['viewCount']))
        else:
            v_views.append(0)
        if 'likeCount' in json_data['items'][v_index]['statistics']:
            v_likes.append(int(json_data['items'][v_index]['statistics']['likeCount']))
        else:
            v_likes.append(0)
        if 'topicDetails' in json_data['items'][v_index] and 'topicCategories' in json_data['items'][v_index]['topicDetails']:
            cleaned_topic_list = list(map(lambda i:get_topic_category_from_wiki(json_data['items'][v_index]['topicDetails']['topicCategories'][i]), range(0, len(json_data['items'][v_index]['topicDetails']['topicCategories']))))
            v_topic_categories.append(cleaned_topic_list)
        else:
            v_topic_categories.append(['NA'])
        if 'commentCount' in json_data['items'][v_index]['statistics']:
            v_comments.append(int(json_data['items'][v_index]['statistics']['commentCount']))
        else:
            v_comments.append(0)
        if 'title' in json_data['items'][v_index]['snippet']:
            v_title.append(json_data['items'][v_index]['snippet']['title'])
        else:
            v_title.append('NA')
        if 'description' in json_data['items'][v_index]['snippet'] and len(json_data['items'][v_index]['snippet']['description']) != 0:
            v_description.append(json_data['items'][v_index]['snippet']['description'])
        else:
            v_description.append('NA')
        if 'publishedAt' in json_data['items'][v_index]['snippet']:
            v_publish_date.append(json_data['items'][v_index]['snippet']['publishedAt'])
        else:
            v_publish_date.append('NA')
        if 'contentDetails' in json_data['items'][v_index] and 'contentRating' in json_data['items'][v_index]['contentDetails']:
            v_contentRating.append(json_data['items'][v_index]['contentDetails']['contentRating'])
        else:
            v_contentRating.append('NA')
        #v_comments.append(getCommentsForVideo(id))
    count = count+1
    print(f'{count}/{videoid_index} processed',end="\r")
yt_data = pd.DataFrame({'videoId': video_id_list,
                        'videoURL': v_url,
                        'videoTitle':v_title,
                        'videoDescription':v_description,
                        'videoPublishedAt':v_publish_date,
                        'channelId':v_channelId,
                         'channelTitle':v_channelTitle,
                         'videoTime':v_videoTime,
                        'videoTags':v_tags,
                        'videoCategory':v_category,
                        'videoTopics':v_topic_categories,
                        'videoLanguage':v_lang,
                        'videoViews':v_views,
                        'videoLikes':v_likes,
                        'videoComments':v_comments,
                        'videoContentRating':v_contentRating,
                       })
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Extracting metadata from YouTube video JSON (w/o comments)','Videos', videoid_index,
                                             dt_string_s,start_time)
yt_data
time_df

Start time: 16:22:55
4371/6421 processedEnd time: 16:22:57d


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,41,16:17:43,16:18:24,1.665703e+09,1.665703e+09
4,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
5,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
6,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
7,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
8,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
9,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09


In [33]:
#Extracting metadata for YouTube channels from JSON

In [34]:
dt_string_s,start_time = startTimeMe(True)
vc_channelTitle = []
vc_topic_categories = []
vc_subs=[]
vc_views=[]
vc_vidcnt=[]
vc_cntry=[]
vc_description=[]
vc_publish_date=[]
vc_visited = []
vc_url = []
data_list = channel_json_data['items']
response_cid_list = []
count = 0
for item in data_list:
    response_cid_list.append(item['id'])
for id in channel_id_list:
    if id != 'NA' and str(id) != 'nan':
        vc_url.append('https://www.youtube.com/c/'+id)
        if id not in response_cid_list:
            vc_channelTitle.append('NA')
            vc_topic_categories.append(['NA'])
            vc_subs.append('NA')
            vc_views.append('NA')
            vc_vidcnt.append('NA')
            vc_cntry.append('NA')
            vc_description.append('NA')
            vc_publish_date.append('NA')
        else:
            v_index = response_cid_list.index(id)
            if 'title' in channel_json_data['items'][v_index]['snippet']:
                vc_channelTitle.append(channel_json_data['items'][v_index]['snippet']['title'])
            else:
                vc_channelTitle.append('NA')
            if 'topicDetails' in channel_json_data['items'][v_index] and 'topicCategories' in channel_json_data['items'][v_index]['topicDetails']:
                cleaned_topic_list = list(
                    map(
                        lambda i:get_topic_category_from_wiki(
                            channel_json_data['items'][v_index]['topicDetails']['topicCategories'][i]), 
                        range(0, len(channel_json_data['items'][v_index]['topicDetails']['topicCategories']))))
                vc_topic_categories.append(cleaned_topic_list)
            else:
                vc_topic_categories.append(['NA'])
            if 'subscriberCount' in channel_json_data['items'][v_index]['statistics']:
                vc_subs.append(int(channel_json_data['items'][v_index]['statistics']['subscriberCount']))
            else:
                vc_subs.append(0)
            if 'viewCount' in channel_json_data['items'][v_index]['statistics']:
                vc_views.append(int(channel_json_data['items'][v_index]['statistics']['viewCount']))
            else:
                vc_views.append(0)
            if 'videoCount' in channel_json_data['items'][v_index]['statistics']:
                vc_vidcnt.append(int(channel_json_data['items'][v_index]['statistics']['videoCount']))
            else:
                vc_vidcnt.append(0)
            if 'country' in channel_json_data['items'][v_index]['snippet']:
                vc_cntry.append(channel_json_data['items'][v_index]['snippet']['country'])
            else:
                vc_cntry.append('NA')
            if 'description' in channel_json_data['items'][v_index]['snippet']:
                vc_description.append(channel_json_data['items'][v_index]['snippet']['description'])
            else:
                vc_description.append('NA')
            if 'publishedAt' in channel_json_data['items'][v_index]['snippet']:
                vc_publish_date.append(channel_json_data['items'][v_index]['snippet']['publishedAt'])
            else:
                vc_publish_date.append('NA')
    count = count+1
    print(f'{count}/{videoid_index} processed',end="\r")
yt_channel_data = pd.DataFrame({'channelTitle':vc_channelTitle,
                                         'channelDescription':vc_description,
                                         'subscribers':vc_subs,
                                         'url':vc_url,
                                        'subscribers':vc_subs,
                                        'channelViews':vc_views,
                                        'videoCount':vc_vidcnt,
                                        'channelCountry':vc_cntry,
                                        'channelTopicCategory':vc_topic_categories,
                                        'channelPublishedAt':vc_publish_date})
dt_string_e,current_time,time_df = endTimeMe(time_df, 'Extracting metadata for YouTube channels from JSON','Channels', channelid_index,
                                             dt_string_s,start_time)
print(time_df)
yt_channel_data

Start time: 16:23:09
End time: 16:23:09
                                             Activity        Type  Quantity  \
0                     Reading 'yturl_twitter' dataset        Rows   1302702   
1                        Reading 'yt_onlyurl' dataset        Rows   1302702   
2   Compiling urls tweet/retweet from twitter dataset        URLs   1302702   
3   Extracting attributes for a date from twitter ...        Rows     18015   
4   Extracting attributes for a date from twitter ...        Rows     18015   
5                    Saving dated twitter data as CSV        URLs     18015   
6                 Reading dated twitter data from CSV        URLs     18015   
7        Segregating ID types from dated twitter data    ID types     18015   
8            Gathering unique video ids from the list  Unique IDs      6450   
9        Generating comma separated list of video ids  Unique IDs      6421   
10     Generating comma separated list of channel ids  Unique IDs        29   
11          

,channelTitle,channelDescription,subscribers,url,channelViews,videoCount,channelCountry,channelTopicCategory,channelPublishedAt
0,Canal Pandemico,Não Há Limites Para Os Sonhos Das Pessoas,178,https://www.youtube.com/c/UC5nItXdQL4fpfW8EBRJ...,28350,1131,NA,[NA],2010-12-13T13:51:47Z
1,Subnet Jobs,Subnet jobs YouTube Channel provide you the hi...,2110,https://www.youtube.com/c/UChwJwruhSicTNnG0Wy5...,131268,951,PK,[NA],2020-12-29T16:20:19.904366Z
2,Keaten Mansfield,Host of Keaten Mansfield Tonight. Join me to d...,394,https://www.youtube.com/c/UCgxzYDehdNgu3491C3h...,11508,52,NA,[NA],2021-08-02T02:01:50.172832Z
3,PEOPLE LIVE 24,PEOPLE LIVE 24 на YouTube - это ежедневная ана...,29000,https://www.youtube.com/c/UC1lokY9bPyxTQx3_EwM...,0,0,RU,[NA],2009-07-15T13:13:06Z
4,NFKRZ,My name is Roman and I'm from Russia. What's up?,1100000,https://www.youtube.com/c/UC19xLluI7dG093Gmw57...,199404984,751,RU,[NA],2010-08-31T14:11:04Z
5,The English Channel Of Al-Mahdi Nasser Mohammed,This channel tackles the translated into Engli...,1860,https://www.youtube.com/c/UCbzTU36Nmiz8UXpdG3R...,71503,101,EG,[NA],2021-09-10T11:26:27.823875Z
6,GBNews,"Freeview 236, Sky HD 515, Virgin Media HD 604,...",573000,https://www.youtube.com/c/UC0vn8ISa4LKMunLbzaX...,308200268,31803,GB,[NA],2021-02-21T14:12:10.972109Z
7,Delta T Racing,If you're interested in league racing on PlayS...,314,https://www.youtube.com/c/UC9cTNDKtwzZppczLzOq...,37638,346,GB,[NA],2021-05-10T09:42:33.783376Z
8,AZOV media,Офіційний канал полку «Азов»\n\nПолк «Азов» од...,159000,https://www.youtube.com/c/UCewl92lzIMDO8QiAYOQ...,35172636,1030,UA,[NA],2014-08-05T17:47:45Z
9,Shakes IV,#speakin' the unspoken in unspeakable tone,81,https://www.youtube.com/c/UCcAbinjQiCbJvlWXQ-Y...,435,4,NA,[NA],2021-10-24T10:27:15.122069Z


In [178]:
#Saving channel and video metadata as per their date

In [174]:
yt_channel_data.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}ChannelDataset.csv',index=False)

In [ ]:
yt_data.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}VideoDataset.csv',index=False)

In [15]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [176]:
yt_channel_data = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}ChannelDataset.csv')
yt_channel_data

,channelTitle,channelDescription,subscribers,url,channelViews,videoCount,channelCountry,channelTopicCategory,channelPublishedAt
0,FUTURE CONFLICT,Future Conflict is a YouTube channel focusing ...,2810,https://www.youtube.com/c/UCD6f-qHBtzBrxnMAeGC...,107929,148,US,['NA'],2006-08-21T02:00:24Z
1,The English Channel Of Al-Mahdi Nasser Mohammed,This channel tackles the translated into Engli...,1850,https://www.youtube.com/c/UCbzTU36Nmiz8UXpdG3R...,70783,101,EG,['NA'],2021-09-10T11:26:27.823875Z
2,Eichhorn Coaching,Alexander Eichhorn und Maximilian Bothe sind e...,5770,https://www.youtube.com/c/UCzEnX97Kzb9G-Zrpl7E...,743072,453,DE,['NA'],2018-02-08T17:18:13Z
3,Gabriele Krone-Schmalz,NaN,3790,https://www.youtube.com/c/UC9bIzo1UsNGjAIiwtAc...,125056,15,NaN,['NA'],2015-09-22T13:35:39Z
4,AZOV media,Офіційний канал полку «Азов»\n\nПолк «Азов» од...,159000,https://www.youtube.com/c/UCewl92lzIMDO8QiAYOQ...,35107430,1030,UA,['NA'],2014-08-05T17:47:45Z
5,Церковь Еммануил,"Христианские проповеди, прославление. Церковь ...",18400,https://www.youtube.com/c/UCu0e3kdu2SrV6bjolfC...,4809713,2783,UA,['NA'],2014-05-23T13:20:13Z
6,Juan1890,NaN,64,https://www.youtube.com/c/UCEvsaEJWYMcjdVJtYNK...,1489,27,NaN,['NA'],2019-09-04T20:18:46Z
7,bald and bankrupt,First Englishman to be made a Thief in Law by ...,3630000,https://www.youtube.com/c/UCxDZs_ltFFvn0FDHT6k...,527564584,240,GB,['NA'],2018-06-12T11:56:25Z
8,TommaHawk79 Games,"Aussie Gamer i Livestream Fifa 20 , 21 & F1 2...",527,https://www.youtube.com/c/UCqO_7yPSaFdHkjf5Bt2...,79819,2177,AU,['NA'],2017-02-07T11:43:33Z
9,Liv Romano,NaN,112,https://www.youtube.com/c/UCZN6grU_dkwYshYQwPi...,1042,21,NaN,['NA'],2021-09-18T01:44:41.25914Z


In [16]:
yt_data = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}VideoDataset.csv')
yt_data

,videoId,videoURL,videoTitle,videoDescription,videoPublishedAt,channelId,channelTitle,videoTime,videoTags,videoCategory,videoTopics,videoLanguage,videoViews,videoLikes,videoComments,videoContentRating
0,eOqZ4ejpSYI,https://www.youtube.com/watch?v=eOqZ4ejpSYI,russian army 2020 الجيش الروسي русская армия,Russian army 2020\nالجيش الروسي 2020,2019-06-26T13:26:03Z,UCfUu8yOs6IXBrROcsl9R30g,السيء,581,['NA'],People & Blogs,"['Military', 'Society']",NaN,3797859,41782,5686,{}
1,aA1npJdN3M4,https://www.youtube.com/watch?v=aA1npJdN3M4,Psaki probed over if sanctions with Russia wil...,White House Press Secretary Jen Psaki has been...,2022-02-23T05:00:50Z,UCO0akufu9MOzyz3nvGIXAAw,Sky News Australia,166,"['6298555357001', 'fb', 'global', 'msn', 'worl...",News & Politics,"['Politics', 'Society']",NaN,8216,195,205,{}
2,5HHrkcXXfWk,https://www.youtube.com/watch?v=5HHrkcXXfWk,ウクライナ人女性が話す【１８歳から日本人男性が好きになった理由４選】,過去にさかのぼって、なぜ日本人の男性に興味を持ったかお話しします。\n全てのお話しは個人意見...,2022-02-22T10:30:01Z,UCdL7FtmjBkgPuroig2UiTJA,クリススタイルチャンネル,870,['NA'],People & Blogs,['Lifestyle_(sociology)'],NaN,94800,3092,108,{}
3,SnoEctYVOWU,https://www.youtube.com/watch?v=SnoEctYVOWU,Putin will stop at nothing to bring back the S...,Eric Bolling exposes Russian President Vladimi...,2022-02-22T22:00:23Z,UCx6h-dWzJ5NpAlja1YsApdg,Newsmax TV,443,"['Newsmax', 'News', 'Politics', 'Conservative'...",News & Politics,"['Politics', 'Society']",NaN,7034,370,232,{}
4,SKSvPaT6V9k,https://www.youtube.com/watch?v=SKSvPaT6V9k,Aida Garifullina & Dmitri Hvorostovsky - Deja ...,Aida Garifullina and Dmitri Hvorostovsky\nNew ...,2015-10-12T09:10:06Z,UCNt7TnGQT0n3dNT83MsR1lA,Aida Garifullina,387,"['Aida Garifullina', 'Dmitri Hvorostovsky', 'n...",Music,"['Classical_music', 'Music', 'Pop_music']",NaN,2578889,23869,1391,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6416,5y_ktAuCypQ,https://www.youtube.com/watch?v=5y_ktAuCypQ,5 reasons Putin's Russia won't have it easy ag...,Play War Thunder for FREE! Register using http...,2022-01-30T22:00:02Z,UCKNCbBWiMiXBVXUmUuu_dsQ,Defense Updates,505,"['ukraine', 'nato', 'putin', 'russia', 'russia...",Science & Technology,"['Military', 'Society']",en,110708,2931,739,{}
6417,opPwDz04kzI,https://www.youtube.com/watch?v=opPwDz04kzI,FULL VIDEO: Trump and Putin Speak After Summit,President Trump and President Vladimir V. Puti...,2018-07-16T16:07:11Z,UCqnbDFdCpuN8CMEg0VuEBqA,The New York Times,5364,"['trump and russia', 'trump and russia speak',...",News & Politics,"['Politics', 'Society']",NaN,73087,550,333,{}
6418,lA3lGR-_sc8,https://www.youtube.com/watch?v=lA3lGR-_sc8,Russia Moves to Recognise Donbass Republics as...,Russia Moves to Recognise Donbass Republics as...,2022-02-21T21:09:58Z,UCwGpHa6rMLjSSCBlckm5khw,Alexander Mercouris,2583,"['Alexander Mercouris', 'The Duran', 'duran ne...",News & Politics,"['Religion', 'Society']",NaN,46494,3552,790,{}
6419,7lrfdzU8k4k,https://www.youtube.com/watch?v=7lrfdzU8k4k,Deadly AC-130 Gunship in Action Firing All Its...,The AC-130 is the world's biggest flying artil...,2018-08-03T15:30:01Z,UCWqPRUsJlZaDp-PVbqEch9g,Smithsonian Channel,188,"['Hell in the sky', 'TV online', 'watch', 'cen...",Entertainment,['Society'],NaN,4906723,68067,4445,{}


In [37]:
time_df = dropRowFrom(time_df,3)
time_df

,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec)
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09
3,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09
4,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09
5,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09
6,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09
7,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09
8,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09
9,Generating comma separated list of channel ids,Unique IDs,29,0,16:21:06,16:21:06,1.665703e+09,1.665703e+09


In [42]:
avg_num = []
totaltime_taken = 0
for idx in time_df.index:
    totaltime_taken = totaltime_taken+time_df['Time Taken (sec)'][idx]
    avg_num.append((time_df['Time Taken (sec)'][idx]/time_df['Quantity'][idx]))
print(f'Total time taken by the script to complete processing for {month_query} {day_query} {year_query}: {totaltime_taken}s')
time_df['TT/Quant'] = avg_num
time_df

Total time taken by the script to complete processing for Feb 23 2022: 190s


,Activity,Type,Quantity,Time Taken (sec),Start Time,End Time,Start (sec),End (sec),TT/Quant
0,Reading 'yturl_twitter' dataset,Rows,1302702,43,16:15:36,16:16:19,1.665703e+09,1.665703e+09,0.000033
1,Reading 'yt_onlyurl' dataset,Rows,1302702,3,16:16:36,16:16:39,1.665703e+09,1.665703e+09,0.000002
2,Compiling urls tweet/retweet from twitter dataset,URLs,1302702,27,16:16:50,16:17:17,1.665703e+09,1.665703e+09,0.000021
3,Extracting attributes for a date from twitter ...,Rows,18015,55,16:18:58,16:19:53,1.665703e+09,1.665703e+09,0.003053
4,Saving dated twitter data as CSV,URLs,18015,0,16:20:30,16:20:30,1.665703e+09,1.665703e+09,0.000000
5,Reading dated twitter data from CSV,URLs,18015,0,16:20:35,16:20:35,1.665703e+09,1.665703e+09,0.000000
6,Segregating ID types from dated twitter data,ID types,18015,0,16:20:46,16:20:46,1.665703e+09,1.665703e+09,0.000000
7,Gathering unique video ids from the list,Unique IDs,6450,0,16:20:53,16:20:53,1.665703e+09,1.665703e+09,0.000000
8,Generating comma separated list of video ids,Unique IDs,6421,0,16:20:59,16:20:59,1.665703e+09,1.665703e+09,0.000000
9,Generating comma separated list of channel ids,Unique IDs,29,0,16:21:06,16:21:06,1.665703e+09,1.665703e+09,0.000000


In [38]:
#Workflow Twitter Processing

In [14]:
file_name = "us-presidential-2020-01-01-00"

In [27]:
twitter_dataset = pd.read_csv(f'./dataset/From Lab/{file_name}.csv')
twitter_dataset

C:\Users\shahm\AppData\Local\Temp\ipykernel_21816\2845688604.py:1: DtypeWarning: Columns (0,1,17,20,22,23,24,25,26,27,43,45,46,47,54,56,57,58,61) have mixed types. Specify dtype option on import or set low_memory=False.
  twitter_dataset = pd.read_csv(f'./dataset/From Lab/{file_name}.csv')


,tweetid,userid,screen_name,date,lang,location,description,place_id,place_url,place_type,...,state,country,rt_state,rt_country,qtd_state,qtd_country,norm_country,norm_rt_country,norm_qtd_country,acc_age
0,1212166581448396801,786863124263505920.0,BlessedMomOf_9,Wed Jan 01 00:20:08 +0000 2020,en,NaN,"Christian, mom of 9 married to their dad for 2...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DC,United States,NaN,NaN,United States,1173.0
1,1212166581704376321,2358613314.0,LorenGardner_,Wed Jan 01 00:20:08 +0000 2020,en,"New York, NY",Podcast Producer/Audio Engineer for @AtWill B...,NaN,NaN,NaN,...,NY,United States,DC,United States,NaN,NaN,United States,United States,NaN,2137.0
2,1212166581813481473,1129196694.0,artsplace421,Wed Jan 01 00:20:08 +0000 2020,en,NaN,ALWAYS TRUMP!🇺🇸🇺🇸🇺🇸🇺🇸#2ndAMD/& NEVER A #SOCIAL...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2528.0
3,1212166581700177921,2290285411.0,PatFaucher1,Wed Jan 01 00:20:08 +0000 2020,en,"Connecticut, USA","Mom, Grandma, cancer survivor - retired & tire...",NaN,NaN,NaN,...,CT,United States,NaN,NaN,NaN,NaN,United States,NaN,NaN,2178.0
4,1212166581775716352,4054001539.0,albert1776,Wed Jan 01 00:20:08 +0000 2020,en,"Houston, TX",Teacher/Scholar/Advocate of American Exception...,NaN,NaN,NaN,...,TX,United States,DC,United States,NaN,NaN,United States,United States,NaN,1524.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70066,1212173397624406017,14206115,Theon_Orbis,Wed Jan 01 00:47:13 +0000 2020,en,NaN,Please pass the popcorn as the Drumpf cult sta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4299.0
70067,1212173397645393921,813473555572670464,whthfk1,Wed Jan 01 00:47:13 +0000 2020,en,NaN,💯🇺🇸Christian Conservative #TrumpTough #Lockher...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100.0
70068,1212173397725048833,2471674532,harpr94,Wed Jan 01 00:47:13 +0000 2020,en,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2071.0
70069,1212173397808963584,1211091474151923712,PennieHendric11,Wed Jan 01 00:47:13 +0000 2020,en,NaN,Brainy,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [32]:
twitter_dataset.columns

Index(['tweetid', 'userid', 'screen_name', 'date', 'lang', 'location',
       'description', 'place_id', 'place_url', 'place_type', 'place_name',
       'place_full_name', 'place_country_code', 'place_country',
       'place_bounding_box', 'text', 'extended', 'coord', 'reply_userid',
       'reply_screen', 'reply_statusid', 'tweet_type', 'friends_count',
       'listed_count', 'followers_count', 'favourites_count', 'statuses_count',
       'verified', 'hashtag', 'urls_list', 'profile_pic_url',
       'profile_banner_url', 'display_name', 'date_first_tweet',
       'account_creation_date', 'rt_urls_list', 'mentionid', 'mentionsn',
       'rt_screen', 'rt_userid', 'rt_user_description', 'rt_text',
       'rt_hashtag', 'rt_qtd_count', 'rt_rt_count', 'rt_reply_count',
       'rt_fav_count', 'rt_tweetid', 'rt_location', 'qtd_screen', 'qtd_userid',
       'qtd_user_description', 'qtd_text', 'qtd_hashtag', 'qtd_qtd_count',
       'qtd_rt_count', 'qtd_reply_count', 'qtd_fav_count', 'qtd_tweeti

In [44]:
dt_string_s,start_time = startTimeMe(True)
tweetid_urldata = []
urllist_urldata = []
for idx in twitter_dataset.index:
    tweetid_urldata.append(twitter_dataset['tweetid'][idx])
    if twitter_dataset['tweet_type'][idx] == 'retweeted_tweet_without_comment':
        urllist_urldata.append(expanded_url_from(twitter_dataset['rt_urls_list'][idx]))
    else:
        urllist_urldata.append(expanded_url_from(twitter_dataset['urls_list'][idx]))

Start time: 00:03:32
Check this link: [{'url': 'https://t.co/EYZy8E47hz', 'expanded_url': 'https://www.thegatewaypundit.com/2019/12/we-have-taken-appropriate-force-protection-actions-to-ensure-safety-of-americans-ah-64-apaches-protect-us-embassy-in-baghdad-video/', 'display_url': 'thegatewaypundit.com/2019/12/we-hav…', 'indices': [94, 117]}]
Check this link: [{'url': 'https://t.co/FuWqXPFAzs', 'expanded_url': 'https://www.thegatewaypundit.com/2019/12/we-have-taken-appropriate-force-protection-actions-to-ensure-safety-of-americans-ah-64-apaches-protect-us-embassy-in-baghdad-video/', 'display_url': 'thegatewaypundit.com/2019/12/we-hav…', 'indices': [137, 160]}]
Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/status/1212139414312275968', 'display_url': 'twitter.com/jackposobiec/s…', 'indices': [89, 112]}]
Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/status/1212139414312275968',

Check this link: [{'url': 'https://t.co/csMYZkvCj3', 'expanded_url': 'https://www.huffpost.com/entry/pete-buttigieg-wine-cave-big-money-warren_n_5e0a4980e4b0b2520d19a336', 'display_url': 'huffpost.com/entry/pete-but…', 'indices': [255, 278]}]
Check this link: [{'url': 'https://t.co/9osGHyeaud', 'expanded_url': 'https://twitter.com/cjgproduxions/status/1212000272848953344?s=20', 'display_url': 'twitter.com/cjgproduxions/…', 'indices': [47, 70]}]
Check this link: [{'url': 'https://t.co/YRFIcVavcg', 'expanded_url': 'http://bit.ly/2QcqL5w', 'display_url': 'bit.ly/2QcqL5w', 'indices': [257, 280]}]
Check this link: [{'url': 'https://t.co/GdvohdblYr', 'expanded_url': 'https://www.washingtonpost.com/opinions/2019/12/31/we-may-have-gotten-down-final-five/', 'display_url': 'washingtonpost.com/opinions/2019/…', 'indices': [228, 251]}]
Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/status/1212139414312275968', 'display_url': 'twitter.com/jack

Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/status/1212139414312275968', 'display_url': 'twitter.com/jackposobiec/s…', 'indices': [89, 112]}]
Check this link: [{'url': 'https://t.co/Plml9lDFzw', 'expanded_url': 'https://twitter.com/adjunctprofesor/status/1212039551960649728', 'display_url': 'twitter.com/adjunctprofeso…', 'indices': [53, 76]}]
Check this link: [{'url': 'https://t.co/pJXv78sMuE', 'expanded_url': 'https://www.cnn.com/2019/12/31/politics/john-roberts-judiciary-democracy/index.html', 'display_url': 'cnn.com/2019/12/31/pol…', 'indices': [175, 198]}]
Check this link: [{'url': 'https://t.co/QKSBecQ977', 'expanded_url': 'https://twitter.com/TomFitton/status/1208130149394333696', 'display_url': 'twitter.com/TomFitton/stat…', 'indices': [87, 110]}]
Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/status/1212139414312275968', 'display_url': 'twitter.com/jackposobiec/s…'

Check this link: [{'url': 'https://t.co/eQDqqMRvq7', 'expanded_url': 'https://twitter.com/teamtrump/status/1212117724626862080', 'display_url': 'twitter.com/teamtrump/stat…', 'indices': [71, 94]}]
Check this link: [{'url': 'https://t.co/yCjRNJVLKX', 'expanded_url': 'https://www.thegatewaypundit.com/2019/12/putin-calls-trump-to-thank-him-for-helping-russia-thwart-terrorist-attack/', 'display_url': 'thegatewaypundit.com/2019/12/putin-…', 'indices': [117, 140]}]
Check this link: [{'url': 'https://t.co/Kb8fkKamX2', 'expanded_url': 'https://www.washingtonpost.com/politics/trump-organization-fires-more-undocumented-workers--a-year-after-its-use-of-illegal-labor-was-revealed/2019/12/31/1f03685a-2b7c-11ea-be79-83e793dbcaef_story.html', 'display_url': 'washingtonpost.com/politics/trump…', 'indices': [182, 205]}]
Check this link: [{'url': 'https://t.co/pJXv78sMuE', 'expanded_url': 'https://www.cnn.com/2019/12/31/politics/john-roberts-judiciary-democracy/index.html', 'display_url': 'cnn.com/2019/

Check this link: [{'url': 'https://t.co/AImxF7S80S', 'expanded_url': 'https://hillreporter.com/so-called-do-nothing-democrats-have-actually-passed-close-to-400-bills-without-gop-led-senate-taking-action-on-them-52351', 'display_url': 'hillreporter.com/so-called-do-n…', 'indices': [144, 167]}]
Check this link: [{'url': 'https://t.co/DEEzU0tXLs', 'expanded_url': 'https://www.thegatewaypundit.com/2019/12/breaking-president-trump-announces-phase-one-in-china-trade-deal-will-take-place-on-january-15/', 'display_url': 'thegatewaypundit.com/2019/12/breaki…', 'indices': [146, 169]}]
Check this link: [{'url': 'https://t.co/A68ueVC88u', 'expanded_url': 'http://45.wh.gov/5yKBD8', 'display_url': '45.wh.gov/5yKBD8', 'indices': [217, 240]}]
Check this link: [{'url': 'https://t.co/A68ueVC88u', 'expanded_url': 'http://45.wh.gov/5yKBD8', 'display_url': '45.wh.gov/5yKBD8', 'indices': [217, 240]}]
Check this link: [{'url': 'https://t.co/qDQtgkjr1S', 'expanded_url': 'https://twitter.com/jackposobiec/statu

Check this link: [{'url': 'https://t.co/TcfwTo5W74', 'expanded_url': 'https://www.washingtonpost.com/politics/how-two-undocumented-housekeepers-took-on-the-president--and-revealed-trumps-long-term-reliance-on-illegal-immigrants/2019/12/04/3dff5b5c-0a15-11ea-bd9d-c628fd48b3a0_story.html', 'display_url': 'washingtonpost.com/politics/how-t…', 'indices': [174, 197]}]
Check this link: [{'url': 'https://t.co/qqEIjSJtvf', 'expanded_url': 'https://twitter.com/ridetwolife/status/1212028675035676672', 'display_url': 'twitter.com/ridetwolife/st…', 'indices': [70, 93]}]
Check this link: [{'url': 'https://t.co/nkkemW6vkO', 'expanded_url': 'https://www.thegatewaypundit.com/2019/12/the-anti-benghazi-president-trump-update-on-baghdad-attack-embassy-is-safe-threatens-iran-over-further-attacks/', 'display_url': 'thegatewaypundit.com/2019/12/the-an…', 'indices': [256, 279]}]
Check this link: [{'url': 'https://t.co/v6GaOEt7hX', 'expanded_url': 'https://www.washingtonpost.com/world/iran-backed-militia-supp

In [48]:
tweetid_list = []
userid_list = []
screenname_list = []
date_list = []
url_list = []
url_id_list = []
url_type_list = []
count_ids = 0
count_cids = 0
count_nonids = 0
count = 0
dt_string_s,start_time = startTimeMe(True)
#tweetid, userid, screen_name, date, url, url_id, url_type
for idx in twitter_dataset.index:
    tweet_id = twitter_dataset['tweetid'][idx]
    if tweet_id in tweetid_urldata:
        tweetid_index = tweetid_urldata.index(tweet_id)
        yt_url = urllist_urldata[tweetid_index]
        if yt_url != "NA" and 'youtube.com' in yt_url or 'youtu.be' in yt_url:
            tweetid_list.append(tweet_id)
            userid_list.append(twitter_dataset['userid'][idx])
            screenname_list.append(twitter_dataset['screen_name'][idx])
            date_list.append(twitter_dataset['date'][idx])
            url_list.append(yt_url)
            link_type, link_id = getIDfromURL(yt_url)
            url_type_list.append(link_type)
            url_id_list.append(link_id)
            if link_type == 'other':
                count_nonids = count_nonids+1
            elif link_type == 'channel':
                count_cids = count_cids+1
            else:
                count_ids = count_ids+1
    count=count+1
    print(f'{count}/{len(twitter_dataset)} processed',end="\r")        
video_twitter_metadata = pd.DataFrame({'tweetid': tweetid_list, 
                                 'userid': userid_list, 
                                 'screen_name': screenname_list, 
                                 'date': date_list,
                                 'url_id': url_id_list,
                                 'url':url_list, 
                                 'url_type':url_type_list })

Start time: 00:09:33
https://www.youtube.com/user/buddymeyers/videos
https://www.youtube.com/user/LionelY2K/live
https://www.youtube.com/embed/Zd9qLfhai90
https://www.youtube.com/user/LionelY2K/live


In [51]:
video_twitter_metadata.to_csv(f'./dataset/Script Extracted/VideoTwitterMetadata_{file_name}.csv',index=False)

In [15]:
video_twitter_metadata = pd.read_csv(f'./dataset/Script Extracted/VideoTwitterMetadata_{file_name}.csv', low_memory=False)

In [18]:
video_twitter_metadata

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1212166587853131777,4.846040e+08,JuliaLeo4,Wed Jan 01 00:20:10 +0000 2020,2lUknzBrT1A,https://youtu.be/2lUknzBrT1A,video
1,1212166622133178370,5.602594e+08,Michg66,Wed Jan 01 00:20:18 +0000 2020,gKByflRb1tg,https://www.youtube.com/watch?v=gKByflRb1tg,video
2,1212166635764641793,1.204208e+18,mypresTrump45,Wed Jan 01 00:20:21 +0000 2020,tU40wWy8ew4,https://youtu.be/tU40wWy8ew4,video
3,1212166665934385153,1.100480e+18,chrisbr56343239,Wed Jan 01 00:20:28 +0000 2020,FLy2duW1wnI,https://www.youtube.com/watch?v=FLy2duW1wnI&t=2s,video
4,1212166699207843841,1.150795e+18,TorontoJon,Wed Jan 01 00:20:36 +0000 2020,dmuyLIrSjxI,https://www.youtube.com/watch?v=dmuyLIrSjxI,video
...,...,...,...,...,...,...,...
287,1212173331149021184,1.087874e+18,DoublePopFun,Wed Jan 01 00:46:57 +0000 2020,8Oea0Dz0w4U,https://youtu.be/8Oea0Dz0w4U,video
288,1212173333833404416,1.187418e+18,bleepfuzz,Wed Jan 01 00:46:58 +0000 2020,FLy2duW1wnI,https://www.youtube.com/watch?v=FLy2duW1wnI&t=2s,video
289,1212173353332682752,1.073287e+18,Hyena1962Dogs,Wed Jan 01 00:47:03 +0000 2020,fMC1I87phjw,https://www.youtube.com/watch?v=fMC1I87phjw,video
290,1212173359213092864,1.671234e+09,mistydawnsmiles,Wed Jan 01 00:47:04 +0000 2020,3wxyN3z9PL4,https://youtu.be/3wxyN3z9PL4,video


In [24]:
video_twitter_metadata['url_type'].unique()

array(['video', 'other', 'channel'], dtype=object)

In [26]:
#Maintaining tracking info table

In [25]:
tracking_info = ''
if exists(f'./dataset/Script Extracted/TrackingInfo.csv'):
    tracking_info = pd.read_csv(f'./dataset/Script Extracted/TrackingInfo.csv')
    vtm_id_list = list(video_twitter_metadata['url_id'])
    vtm_id_type = list(video_twitter_metadata['url_type'])
    ti_id_list = list(tracking_info['ID'])
    ti_new_id = []
    ti_new_row_idx = []
    count = 0
    for idx, vtm_id_item in enumerate(vtm_id_list):
        if vtm_id_item not in ti_id_list and vtm_id_type[idx] != 'other':
            ti_new_id.append(vtm_id_item)
            ti_new_row_idx.append(idx)
            count = count+1
    print(f'{count} new ids added to be tracked')
else:
    today = date.today()
    today_dt = today.strftime("%d/%m/%Y")
    ti_id = []
    ti_url = []
    ti_ts = []
    ti_lt = []
    ti_videoflag = []
    ti_status = []
    for idx in video_twitter_metadata.index:
        if video_twitter_metadata['url_type'][idx] == 'video' or video_twitter_metadata['url_type'][idx] == 'channel':
            ti_id.append(video_twitter_metadata['url_id'][idx])
            ti_url.append(video_twitter_metadata['url'][idx])
            ti_ts.append(today_dt)
            ti_lt.append(today_dt)
            if video_twitter_metadata['url_type'][idx] == 'video':
                ti_videoflag.append(True)
            else:
                ti_videoflag.append(False)
            ti_status.append('Monitored')
    tracking_info = pd.DataFrame({'ID': ti_id, 
                                 'URL': ti_url, 
                                 'tracked_since': ti_ts, 
                                 'last_tracked': ti_lt,
                                 'is_video': ti_videoflag,
                                 'status':ti_status })
    tracking_info
    tracking_info.to_csv(f'./dataset/Script Extracted/TrackingInfo.csv',index=False)

0 new ids added to be tracked


In [ ]:
tracking_info

In [28]:
twitter_dataset.columns

Index(['tweetid', 'userid', 'screen_name', 'date', 'lang', 'location',
       'description', 'place_id', 'place_url', 'place_type', 'place_name',
       'place_full_name', 'place_country_code', 'place_country',
       'place_bounding_box', 'text', 'extended', 'coord', 'reply_userid',
       'reply_screen', 'reply_statusid', 'tweet_type', 'friends_count',
       'listed_count', 'followers_count', 'favourites_count', 'statuses_count',
       'verified', 'hashtag', 'urls_list', 'profile_pic_url',
       'profile_banner_url', 'display_name', 'date_first_tweet',
       'account_creation_date', 'rt_urls_list', 'mentionid', 'mentionsn',
       'rt_screen', 'rt_userid', 'rt_user_description', 'rt_text',
       'rt_hashtag', 'rt_qtd_count', 'rt_rt_count', 'rt_reply_count',
       'rt_fav_count', 'rt_tweetid', 'rt_location', 'qtd_screen', 'qtd_userid',
       'qtd_user_description', 'qtd_text', 'qtd_hashtag', 'qtd_qtd_count',
       'qtd_rt_count', 'qtd_reply_count', 'qtd_fav_count', 'qtd_tweeti

In [37]:
print('Stats on # of original tweets, # of replies, # of retweet, # of quotes')
print(twitter_dataset['tweet_type'].value_counts())

Stats on # of original tweets, # of replies, # of retweet, # of quotes
retweeted_tweet_without_comment    33545
reply                              17061
quoted_tweet                       10825
original                             998
False                                  8
-0.7845                                2
0.8313                                 2
0.3089                                 1
United States                          1
Name: tweet_type, dtype: int64


In [40]:
print('Stats on # of users, and displaying the users_list')
print(twitter_dataset['userid'].value_counts())
print(list(twitter_dataset['userid'].unique()))

Stats on # of users, and displaying the users_list
95332088                  7617
1110304448868433920         51
1357519957                  40
1.0844835947335393e+18      40
704047778037534720          39
                          ... 
20342960                     1
1208022186642026496          1
3572872281                   1
973304189328687104           1
813473555572670464           1
Name: userid, Length: 33487, dtype: int64
[7.868631242635059e+17, 2358613314.0, 1129196694.0, 2290285411.0, 4054001539.0, 2637980849.0, 8.228103526100828e+17, 1978165356.0, 312302456.0, 8.177990703161344e+17, 1.1415399018479821e+18, 1.1076476846121861e+18, 316593845.0, 8.239559996985221e+17, 1.159828432642949e+18, 1.1938644437777572e+18, 8.044354074000957e+17, 845848194.0, 1.198052059779453e+18, 1.1932403478764298e+18, 17866843.0, 354022572.0, 1.0869761308198093e+18, 8.570876590088151e+17, 43870406.0, 621110959.0, 1.0425945811439903e+18, 8.964872715490673e+17, 33463746.0, 8.24405067755352e+17, 1.20423

In [39]:
#Workflow YouTube static metadata

In [45]:
if exists(f'./dataset/Script Extracted/VideoStaticMetadata.csv'):
    video_static_metadata = pd.read_csv(f'./dataset/Script Extracted/VideoStaticMetadata.csv',low_memory)
else:
    ti_id = list(tracking_info['ID'])
    ti_status = list(tracking_info['status'])
    ti_videoflag = list(tracking_info['is_video'])
    vsm_id = []
    count_vids = 0
    for idx,ti_id_item in enumerate(ti_id):
        #print(f'{ti_status[idx]} {ti_videoflag[idx]}')
        if ti_status[idx] == 'Monitored' and ti_videoflag[idx] == True:
            vsm_id.append(ti_id_item)
            count_vids=count_vids+1
    print(f'{count_vids} videos to be tracked')

286 videos to be tracked


In [46]:
if exists(f'./dataset/Script Extracted/ChannelStaticMetadata.csv'):
    video_static_metadata = pd.read_csv(f'./dataset/Script Extracted/ChannelStaticMetadata.csv',low_memory)
else:
    ti_id = list(tracking_info['ID'])
    ti_status = list(tracking_info['status'])
    ti_videoflag = list(tracking_info['is_video'])
    csm_id = []
    count_chls = 0
    for idx,ti_id_item in enumerate(ti_id):
        #print(f'{ti_status[idx]} {ti_videoflag[idx]}')
        if ti_status[idx] == 'Monitored' and ti_videoflag[idx] == False:
            csm_id.append(ti_id_item)
            count_chls=count_chls+1
    print(f'{count_chls} channels to be tracked')

2 channels to be tracked
